In [5]:

import acquire as a
from env import github_token, github_username
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import scipy.stats as stats
from scipy.stats import  chi2_contingency
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import ttest_ind
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import json
from typing import Dict, List, Optional, Union, cast



In [6]:
REPOS = []
 

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

In [7]:
url = 'https://github.com/search?o=desc&q=stars%3A%3E1&s=forks&type=Repositories&p=1'

In [8]:
def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data

In [9]:
df = github_api_request(url)

In [10]:
df

{'payload': {'header_redesign_enabled': False,
  'results': [{'id': '2325298',
    'archived': False,
    'color': '#555555',
    'followers': 155958,
    'has_funding_file': False,
    'hl_name': 'torvalds/linux',
    'hl_trunc_description': 'Linux kernel source tree',
    'language': 'C',
    'mirror': False,
    'owned_by_organization': False,
    'public': True,
    'repo': {'repository': {'id': 2325298,
      'name': 'linux',
      'owner_id': 1024025,
      'owner_login': 'torvalds',
      'updated_at': '2023-08-17T14:59:13.539Z',
      'has_issues': False}},
    'sponsorable': False,
    'topics': [],
    'type': 'Public',
    'help_wanted_issues_count': 0,
    'good_first_issue_issues_count': 0,
    'starred_by_current_user': False},
   {'id': '63476337',
    'archived': False,
    'color': '#3572A5',
    'followers': 164812,
    'has_funding_file': True,
    'hl_name': 'TheAlgorithms/Python',
    'hl_trunc_description': 'All Algorithms implemented in Python',
    'language': '

In [35]:
repo = 'python/cpython'

In [36]:
def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


In [37]:
df = get_repo_contents(repo)

In [38]:
df

[{'name': '.azure-pipelines',
  'path': '.azure-pipelines',
  'sha': '007ccefbe4d1b550c3f1f8ecb295270217bc6112',
  'size': 0,
  'url': 'https://api.github.com/repos/python/cpython/contents/.azure-pipelines?ref=main',
  'html_url': 'https://github.com/python/cpython/tree/main/.azure-pipelines',
  'git_url': 'https://api.github.com/repos/python/cpython/git/trees/007ccefbe4d1b550c3f1f8ecb295270217bc6112',
  'download_url': None,
  'type': 'dir',
  '_links': {'self': 'https://api.github.com/repos/python/cpython/contents/.azure-pipelines?ref=main',
   'git': 'https://api.github.com/repos/python/cpython/git/trees/007ccefbe4d1b550c3f1f8ecb295270217bc6112',
   'html': 'https://github.com/python/cpython/tree/main/.azure-pipelines'}},
 {'name': '.coveragerc',
  'path': '.coveragerc',
  'sha': 'b5d94317e8aa8b0a06f1c2ff2e96e31c091ed992',
  'size': 526,
  'url': 'https://api.github.com/repos/python/cpython/contents/.coveragerc?ref=main',
  'html_url': 'https://github.com/python/cpython/blob/main/.c

In [47]:
import requests
from bs4 import BeautifulSoup


url = 'https://github.com/python/cpython/blob/main/README.rst'


response = requests.get(url)

if response.status_code == 200:
    
    soup = BeautifulSoup(response.text, 'html.parser')

    readme_content = soup.get_text()
    

    data = {'URL': [url], 'Content': [readme_content]}
    df = pd.DataFrame(data)

In [48]:
df

,URL,Content
0,https://github.com/python/cpython/blob/main/RE...,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."


In [49]:
df.Content.value_counts()

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":".azure-pipelines","path":".azure-pipelines","contentType":"directory"},{"name":".devcontainer","path":".devcontainer","contentType":"directory"},{"name":".github","path":".github","contentType":"directory"},{"name":"Doc","path":"Doc","contentType":"directory"},{"name":"Grammar","path":"Grammar","contentType":"directory"},{"name":"Include","path":"Include","contentType":"directory"},{"name":"Lib","path":"Lib","contentType":"directory"},{"name":"Mac","path":"Mac","contentType":"directory"},{"name":"Misc","path":"Misc","contentType":"directory"},{"name":"Modules","path":"Modules","contentType":"directory"},{"name":"Objects","path":"Objects","contentType":"directory"},{"name":"PC","path":"PC","contentType":"directory"},{"name":"PCbuild","path":"PCbuild","contentType":"directory"},{"name":"Parser","path":"Parser","contentType":"directory"},{"name":"Programs","path":"Programs","contentType":"directory"},{"name":"Python

In [50]:
pattern = r'"richText":"(.*?)"\s*,\s*"renderedFileInfo"'

matches = re.findall(pattern, readme_content)

if matches:
    extracted_text = matches[0]
    print(extracted_text)
else:
    print("No matching content found.")

This is Python version 3.13.0 alpha 0\n\n\n\n\n\nCopyright © 2001-2023 Python Software Foundation.  All rights reserved.\nSee the end of this file for further copyright and license information.\n\nContents\n\nGeneral Information\nContributing to CPython\nUsing Python\nBuild Instructions\nProfile Guided Optimization\nLink Time Optimization\n\n\nWhat's New\nDocumentation\nConverting From Python 2.x to 3.x\nTesting\nInstalling multiple versions\nRelease Schedule\nCopyright and License Information\n\n\n\nGeneral Information\n\nWebsite: https://www.python.org\nSource code: https://github.com/python/cpython\nIssue tracker: https://github.com/python/cpython/issues\nDocumentation: https://docs.python.org\nDeveloper's Guide: https://devguide.python.org/\n\n\nContributing to CPython\nFor more complete instructions on contributing to CPython development,\nsee the Developer Guide.\n\nUsing Python\nInstallable Python kits, and information about using Python, are available at\npython.org.\n\nBuild I

In [51]:
def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )

In [52]:
lang = get_repo_language(repo)

In [53]:
lang

'Python'